In [32]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import json
import os

data_path = Path('.').resolve() / 'videos'

In [31]:
def rename_files(data_path):
    x = dict()
    for i, (p, name, kor_name) in enumerate(zip(
            data_path.glob('*.mp4'), 
            ['hipjoint', 'shoulder', 'waist', 'trapezius'], 
            ['고관절', '어깨', '허리디스크', '승모근']
        )):
        pname = p.name.rstrip('.mp4')
        new_name = f'{i:02d}_{name}'
        x[new_name] = {'video_name': pname, 'kor_name': kor_name}
        p.rename(data_path / f'{new_name}.mp4')
    with (data_path / 'video_names.json').open('w') as file:
        json.dump(x, file)

* 영상을 살펴보니 00_hipjoint는 두 개의 동작이 있음
* 각 영상에는 설명부분과 운동하는 부분이 있음
* 한 부위에 대해서 두 가지 방법을 알려주기도 함

In [ ]:
import cv2

: 